<a href="https://colab.research.google.com/github/rmicallef/hd-experiments/blob/main/GPT_2uning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goals:

Finetune GPT-2 on per-environment (i.e., genre) fan fiction corpus, and use hand-written metascenes to generate candidates

Questions: How well does this work? What's the quality of the generated output?

How specifically do we need to fine-tune the corpus? Can a single generator work for multiple genres?

How do we use transformers here?

Let's see what DialoGPT looks like. I'm not sure it's going to be usefulfor anything other than NPC dialog, but it's worth seeing how well it holds up to button mashing.

In [ ]:
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
cd transformers

/content/transformers


In [ ]:
# this will install packages that require restarting the kernel, so it will end up needing to be run again after kernel restart

!pip install .
#!pip install -r ./examples/requirements.txt
#!pip install -I pyarrow==0.17.1 numpy==1.18.0 future==0.18.1

Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.1.0.dev0-cp36-none-any.whl size=1448430 sha256=e73bccf8520a59df86648ddc323e5a821025704f942279dfa3efde9bffc35c72
  Stored in directory: /tmp/pip-ephem-wheel-cache-h2t9__mq/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 4.1.0.dev0
    Uninstalling transformers-4.1.0.dev0:
      Successfully uninstalled transformers-4.1.0.dev0


In [ ]:
# DialoGPT Tests

'''
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")


# Let's chat for a few lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> ") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("God: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
'''

'\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\nimport torch\n\ntokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")\nmodel = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")\n\n\n# Let\'s chat for a few lines\nfor step in range(5):\n    # encode the new user input, add the eos_token and return a tensor in Pytorch\n    new_user_input_ids = tokenizer.encode(input(">> ") + tokenizer.eos_token, return_tensors=\'pt\')\n\n    # append the new user input tokens to the chat history\n    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids\n\n    # generated a response while limiting the total chat history to 1000 tokens, \n    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)\n\n    # pretty print last ouput tokens from bot\n    print("God: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_token

In [ ]:
# Hardware lottery!
!nvidia-smi

Fri Dec 11 15:12:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# No dialog for now, just generation. Less talk, more rokk 
# Generate text with untuned model
# Here, we'll use the command line version

prompt = '"The quick brown fox"'
temp = '0.8'
num_return_sequences = '3'

!python ./examples/text-generation/run_generation.py --model_type=gpt2 --model_name_or_path=gpt2 --prompt={prompt} --length=50 --temperature={temp} --num_return_sequences={num_return_sequences}

2020-12-11 15:13:04.304569: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/11/2020 15:13:09 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/11/2020 15:13:44 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=50, model_name_or_path='gpt2', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=3, p=0.9, padding_text='', prefix='', prompt='The quick brown fox', repetition_penalty=1.0, seed=42, stop_token=None, temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
The quick brown foxes take advantage of this and lead the prey away with a clever move.

This is an excellent time to take a close look at the prey, as they can make a good bait. The foxes are very protective of their prey, especially
=== GENERATED SEQUENCE 2 ===
The quick brown foxes follow the raised paws 

In [ ]:
# Generate text with the code from inside the huggingface example

import argparse
import logging

import numpy as np
import torch

from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

MODEL_CLASSES = {
    "gpt2": (GPT2LMHeadModel, GPT2Tokenizer),
    "ctrl": (CTRLLMHeadModel, CTRLTokenizer),
    "openai-gpt": (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    "xlnet": (XLNetLMHeadModel, XLNetTokenizer),
    "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
    "xlm": (XLMWithLMHeadModel, XLMTokenizer),
}

# Padding text to help Transformer-XL and XLNet with short prompts as proposed by Aman Rusia
# in https://github.com/rusiaaman/XLNet-gen#methodology
# and https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e
PREFIX = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""


def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

#
# Functions to prepare models' input
#


def prepare_ctrl_input(args, _, tokenizer, prompt_text):
    if args.temperature > 0.7:
        logger.info("CTRL typically works better with lower temperatures (and lower top_k).")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
        logger.info("WARNING! You are not starting your generation from a control code so you won't get good results")
    return prompt_text


def prepare_xlm_input(args, model, tokenizer, prompt_text):
    # kwargs = {"language": None, "mask_token_id": None}

    # Set the language
    use_lang_emb = hasattr(model.config, "use_lang_emb") and model.config.use_lang_emb
    if hasattr(model.config, "lang2id") and use_lang_emb:
        available_languages = model.config.lang2id.keys()
        if args.xlm_language in available_languages:
            language = args.xlm_language
        else:
            language = None
            while language not in available_languages:
                language = input("Using XLM. Select language in " + str(list(available_languages)) + " >>> ")

        model.config.lang_id = model.config.lang2id[language]
        # kwargs["language"] = tokenizer.lang2id[language]

    # TODO fix mask_token_id setup when configurations will be synchronized between models and tokenizers
    # XLM masked-language modeling (MLM) models need masked token
    # is_xlm_mlm = "mlm" in args.model_name_or_path
    # if is_xlm_mlm:
    #     kwargs["mask_token_id"] = tokenizer.mask_token_id

    return prompt_text


def prepare_xlnet_input(args, _, tokenizer, prompt_text):
    prefix = args.prefix if args.prefix else args.padding_text if args.padding_text else PREFIX
    prompt_text = prefix + prompt_text
    return prompt_text


def prepare_transfoxl_input(args, _, tokenizer, prompt_text):
    prefix = args.prefix if args.prefix else args.padding_text if args.padding_text else PREFIX
    prompt_text = prefix + prompt_text
    return prompt_text


PREPROCESSING_FUNCTIONS = {
    "ctrl": prepare_ctrl_input,
    "xlm": prepare_xlm_input,
    "xlnet": prepare_xlnet_input,
    "transfo-xl": prepare_transfoxl_input,
}


def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length


In [ ]:
### Main() from huggingface example code with a few modifications

def remember_the_main(args_to_pass):
    
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_type",
        default=None,
        type=str,
        required=True,
        help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--model_name_or_path",
        default=None,
        type=str,
        required=True,
        help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )

    parser.add_argument("--prompt", type=str, default="")
    parser.add_argument("--length", type=int, default=20)
    parser.add_argument("--stop_token", type=str, default=None, help="Token at which text generation is stopped")

    parser.add_argument(
        "--temperature",
        type=float,
        default=1.0,
        help="temperature of 1.0 has no effect, lower tend toward greedy sampling",
    )
    parser.add_argument(
        "--repetition_penalty", type=float, default=1.0, help="primarily useful for CTRL model; in that case, use 1.2"
    )
    parser.add_argument("--k", type=int, default=0)
    parser.add_argument("--p", type=float, default=0.9)

    parser.add_argument("--prefix", type=str, default="", help="Text added prior to input.")
    parser.add_argument("--padding_text", type=str, default="", help="Deprecated, the use of `--prefix` is preferred.")
    parser.add_argument("--xlm_language", type=str, default="", help="Optional language when used with the XLM model.")

    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument("--num_return_sequences", type=int, default=1, help="The number of samples to generate.")
    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    parser.add_argument('--verbosity', type=int, default=3, help='how chatty do you want your system output')

    # modify the defaults with our own list
    args = parser.parse_args(args_to_pass)

    args.device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()

    if args.verbosity >= 2: logger.info(
        "device: %s, n_gpu: %s, 16-bits training: %s",
        args.device,
        args.n_gpu,
        args.fp16,
    )

    set_seed(args)

    # BUSINESS TIME
    # Initialize the model and tokenizer
    try:
        args.model_type = args.model_type.lower()
        model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    except KeyError:
        raise KeyError("the model {} you specified is not supported. You are welcome to add it and open a PR :)")

    tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path)
    model = model_class.from_pretrained(args.model_name_or_path)
    model.to(args.device)

    if args.fp16:
        model.half()

    args.length = adjust_length_to_model(args.length, max_sequence_length=model.config.max_position_embeddings)
    if args.verbosity >= 3: logger.info(args)

    prompt_text = args.prompt if args.prompt else input("Model prompt >>> ")

    # Different models need different input formatting and/or extra arguments
    requires_preprocessing = args.model_type in PREPROCESSING_FUNCTIONS.keys()
    if requires_preprocessing:
        prepare_input = PREPROCESSING_FUNCTIONS.get(args.model_type)
        preprocessed_prompt_text = prepare_input(args, model, tokenizer, prompt_text)

        if model.__class__.__name__ in ["TransfoXLLMHeadModel"]:
            tokenizer_kwargs = {"add_space_before_punct_symbol": True}
        else:
            tokenizer_kwargs = {}

        encoded_prompt = tokenizer.encode(
            preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", **tokenizer_kwargs
        )
    else:
        prefix = args.prefix if args.prefix else args.padding_text
        encoded_prompt = tokenizer.encode(prefix + prompt_text, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(args.device)

    if encoded_prompt.size()[-1] == 0:
        input_ids = None
    else:
        input_ids = encoded_prompt

    output_sequences = model.generate(input_ids=input_ids,    max_length=args.length + len(encoded_prompt[0]),    temperature=args.temperature,    top_k=args.k,    top_p=args.p,    repetition_penalty=args.repetition_penalty,    do_sample=True,    num_return_sequences=args.num_return_sequences,)

    # Remove the batch dimension when returning multiple sequences
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        if args.verbosity >= 3: print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1)) 
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(args.stop_token) if args.stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
        if args.verbosity >= 1: print(total_sequence)

    return generated_sequences

In [ ]:
# Text generation harness

def format_args(args_dict):
    args_list = []
    for k, v in args_dict.items():
        args_list.append('--{}'.format(k))
        args_list.append(str(v))
    return args_list

def generate_text(prompts, models, args_dict):

    for prompt in prompts:
        args_dict['prompt'] = prompt
        print('\n\nPROMPT: {}\n'.format(prompt))

        for model in models:
            args_dict['model_name_or_path'] = model
            
            generated_sequences = remember_the_main(format_args(args_dict))

            for i, sequence in enumerate(generated_sequences):
                print('\n** Model {}, seq {}:\n{}\n**'.format(model, i, sequence))

In [ ]:
# Now we use the inline function instead of the command line

default_args_dict = {
    'num_return_sequences': 1,
    'length': 30,
    'temp': 1.0,
    'verbosity': 0,
    'model_type': 'gpt2', #for now, we're just looking at GPT-2 based models
}

generate_text(['The quick brown fox'], ['gpt2'], default_args_dict)



PROMPT: The quick brown fox



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
The quick brown foxes you see aren't as big, but they have a hard time making it hard to get on with the grass up ahead. Once they get over
**


In [ ]:
# Let's vary the prompts a little

test_prompts = [
    'Harry opened the door and saw',
    'The New York Times reported today that',
    'The Daily Prophet reported today that',
    "Picard stepped off the shuttlecraft. He couldn't believe his eyes",
]

generate_text(test_prompts, ['gpt2'], default_args_dict)



PROMPT: Harry opened the door and saw



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Harry opened the door and saw none of the bandits who had waited in the street. They had been making their way back home on foot to help up the Auctie.

**


PROMPT: The New York Times reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
The New York Times reported today that for the past four years, the Chief Justice had suggested that some individuals making outrageous and unconstitutional anti-Semitic statements should be expelled from the Supreme Court.
**


PROMPT: The Daily Prophet reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
The Daily Prophet reported today that for the past four years, the Chief of Staff of the Supreme Government has been portrayed as a "courier for all her colleagues, the most
**


PROMPT: Picard stepped off the shuttlecraft. He couldn't believe his eyes



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Picard stepped off the shuttlecraft. He couldn't believe his eyes, the station was in such awful condition. "It would have been wonderful to have seen you on time."

The AOC looked at him
**


In [ ]:
# Now we tune model with a training corpus

!python /content/transformers/examples/language-modeling/run_clm.py \
--output_dir=/content/models/gpt2-potter-micro-8epochs \
--model_type=gpt2 \
--model_name_or_path=/content/models/gpt2-potter-micro-3epochs \
--do_train \
--train_file=/content/data/potter-train-micro.txt \
--do_eval \
--validation_file=/content/data/potter-test-micro.txt \
--per_device_train_batch_size=1 \
--per_device_eval_batch_size=1 \
--learning_rate 5e-5 \
--num_train_epochs=5

2020-12-11 15:14:35.596535: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/11/2020 15:14:37 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/11/2020 15:14:37 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/models/gpt2-potter-micro-8epochs', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec11_15-14-37_82d91dacce5

In [ ]:
# Generate text with a tuned model

generate_text(test_prompts, ['/content/models/gpt2-potter-micro-1epoch'], default_args_dict)



PROMPT: Harry opened the door and saw



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1epoch, seq 0:
Harry opened the door and saw none of the Slytherins in the hall. He stopped in front of Harry and looked at Ginny with admiration. She was her usual face. �
**


PROMPT: The New York Times reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1epoch, seq 0:
The New York Times reported today that for the past four years, the Chief Justice had suggested that some individuals making it hard for human beings to live would be forced to live outside the homes
**


PROMPT: The Daily Prophet reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1epoch, seq 0:
The Daily Prophet reported today that none of the 21 grandchildren who survived the fire had any memory of their grandfathers fall. The burning remains of the home were found by security at the
**


PROMPT: Picard stepped off the shuttlecraft. He couldn't believe his eyes



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1epoch, seq 0:
Picard stepped off the shuttlecraft. He couldn't believe his eyes, the station was in such awful condition. Sirius and the others were making it hard not to let things like that slip by. Once, the crew
**


In [15]:
# Now let's compare some models.

test_prompts = [
    'Harry opened the door and saw',
    #'What have you done with my wand?',
    'The New York Times reported today that',
    'The Daily Prophet reported today that',
    #'There was trouble afoot at Hogwarts yesterday.',
    #'There was trouble afoot at Harvard yesterday.',
    #'There was trouble afoot at Kennedy High School yesterday.'
    #'Hermione was quite excited about tomorrow.',
    #'Melissa was quite excited about tomorrow.',
    "Picard stepped off the shuttlecraft. He couldn't believe his eyes",
]

models = [
          'gpt2',
          #'/content/models/gpt2-potter-micro-1epoch',
          #'/content/models/gpt2-potter-micro-2epochs',
          #'/content/models/gpt2-potter-micro-3epochs',
          '/content/models/gpt2-potter-micro-8epochs',
]

generate_text(test_prompts, models, default_args_dict)



PROMPT: Harry opened the door and saw



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Harry opened the door and saw none of the bandits who had waited in the street. They had been making their way back home on foot to help up the Auctie.

**


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-8epochs, seq 0:
Harry opened the door and saw Mr. Weasley walking in. Harry turned and saw the cloaked figure walking towards him. ‘What’s the matter?’ 
**


PROMPT: The New York Times reported today that



...


** Model /content/models/gpt2-potter-micro-8epochs, seq 0:
The Daily Prophet reported today that none of the four members of the caliphate are being held in secret, except for Mokhtar al-Shehri, who will face trial this
**


PROMPT: Picard stepped off the shuttlecraft. He couldn't believe his eyes



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Picard stepped off the shuttlecraft. He couldn't believe his eyes, the station was in such awful condition. "It would have been wonderful to have seen you on time."

The AOC looked at him
**


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-8epochs, seq 0:
Picard stepped off the shuttlecraft. He couldn't believe his eyes, the station was utterly alien. He'd had enough. When he was announced to be at sea, the sounds of all her passengers squawking
**


In [1]:
# Note that we're training on full and evaluating on micro - this won't hurt the output, just speed up the perplexity measurement at the cost of a little perplecity accuracy
# JK, training on full and testing on micro risks overlap between the training set and the test set, which taints the perplexity score
!python /content/transformers/examples/language-modeling/run_clm.py \
--output_dir=/content/models/gpt2-potter-full-3epochs \
--model_type=gpt2 \
--model_name_or_path=/content/models/gpt2-potter-full-1epoch \
--do_train \
--train_file=/content/data/potter-train-full.txt \
--do_eval \
--validation_file=/content/data/potter-test-full.txt \
--per_device_train_batch_size=1 \
--per_device_eval_batch_size=1 \
--learning_rate 5e-5 \
--num_train_epochs=2

2020-12-11 17:40:51.669781: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/11/2020 17:40:57 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/11/2020 17:40:57 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/models/gpt2-potter-full-3epochs', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec11_17-40-57_82d91dacce52

In [18]:
#DistilGPT2

!python /content/transformers/examples/language-modeling/run_clm.py \
--output_dir=/content/models/distilgpt2-potter-full-1epoch \
--model_type=gpt2 \
--model_name_or_path=distilgpt2 \
--do_train \
--train_file=/content/data/potter-train-full.txt \
--do_eval \
--validation_file=/content/data/potter-test-full.txt \
--per_device_train_batch_size=1 \
--per_device_eval_batch_size=1 \
--learning_rate 5e-5 \
--num_train_epochs=1

2020-12-11 16:56:37.128284: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/11/2020 16:56:42 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/11/2020 16:56:42 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/models/distilgpt2-potter-full-1epoch', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec11_16-56-42_82d91da

In [ ]:
models = [
          'gpt2',
          #'/content/models/gpt2-potter-micro-1epoch',
          #'/content/models/gpt2-potter-micro-2epochs',
          #'/content/models/gpt2-potter-micro-3epochs',
          '/content/models/gpt2-potter-micro-8epochs',
          '/content/models/gpt2-potter-full-1epoch',
          '/content/models/distilgpt2-potter-full-1epoch'
]

generate_text(test_prompts, models, default_args_dict)



PROMPT: Harry opened the door and saw



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Harry opened the door and saw none of the bandits who had waited in the street. They had been making their way back home on foot to help up the Auctie.

**


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-8epochs, seq 0:
Harry opened the door and saw Mr. Weasley walking in. Harry turned and saw the cloaked figure walking towards him. ‘What’s the matter?’ 
**


...


** Model /content/models/distilgpt2-potter-full-1epoch, seq 0:
Harry opened the door and saw James sit down and clasped Harry’s arm. “James,” he said with a smirk. “He’
**


PROMPT: The New York Times reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
The New York Times reported today that for the past four years, the Chief Justice had suggested that some individuals making outrageous and unconstitutional anti-Semitic statements should be expelled from the Supreme Court.
**


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-8epochs, seq 0:
The New York Times reported today that for the second straight year, the Chief Justice had suggested that some members of his court could be forced to live with a mentally ill spouse, even as
**


In [ ]:
'''
Notes:
- tuning gpt2 on potter-micro (107 texts) takes about 6 minutes per epoch on a Tesla K80
- tuning gpt2 on potter-micro (107 texts) takes about 50 minutes per epoch without a GPU
- tuning gpt2 on potter-full (807 texts) takes about 50 minutes per epoch on a Tesla K80 - 1-epoch perplexity 18.88
- tuning distilgpt2 on potter-full (807 texts) takes about 39 minutes per epoch on a Tesla K80 - 1-epoch perplexity 27.00


Next steps
- address the padding warning
- try another corpus
- try another model family?
- try a template?
'''